In [1]:
from RestrictedPython import compile_restricted
from RestrictedPython import safe_builtins
from RestrictedPython import utility_builtins

from allowed import lsst_builtins  # Whitelisted modules

In [2]:
safe_locals = {'__builtins__': {**lsst_builtins, **safe_builtins, **utility_builtins}}

## Define a function to run a filter in restricted mode.

In [3]:
def runFilterRestricted(source_code):
    byte_code = compile_restricted(
        source_code,
        filename='<inline code>',
        mode='exec'
        )
    loc={}

    exec(byte_code, safe_locals, loc)

    return(loc['run_filter']())

## Define a function to run a filter unrestricted.
To show that the function is generally valid for comparison.

In [4]:
def runFilterUnrestricted(source_code):
    byte_code = compile(
        source_code,
        filename='<inline code>',
        mode='exec'
        )
    loc={}

    exec(byte_code, globals(), loc)

    return(loc['run_filter']())

## Capabilities we may want to block

* Using os or other imported packages
* Looking for local files
* Executing files
* Opening local files
* Writing to files on disk
* Writing to stdout

#### os

In [5]:
test_os = """
def run_filter():
    import os
    return (os.listdir()) 
"""

Test the above function works without RestrictedPython enabled:

In [6]:
print(runFilterUnrestricted(test_os))

['setup_profile', 'filter1.py', 'Dockerfile', 'Dockerfile-python', 'pandas-test.csv', 'allowed.py', 'explore-restricted-python.ipynb', '__pycache__', 'junk.py', 'example.py', 'hello-world.py', '.ipynb_checkpoints', 'run_setup.sh', 'test-run.py', 'output.txt', 'stamp-676.fits']


Test with RestrictedPython enabled:

In [7]:
print(runFilterRestricted(test_os))

ImportError: __import__ not found

#### glob

In [8]:
test_glob = """
def run_filter():
    import glob  
    return(glob.glob("*"))
"""

Unrestricted:

In [9]:
print(runFilterUnrestricted(test_glob))

['setup_profile', 'filter1.py', 'Dockerfile', 'Dockerfile-python', 'pandas-test.csv', 'allowed.py', 'explore-restricted-python.ipynb', '__pycache__', 'junk.py', 'example.py', 'hello-world.py', 'run_setup.sh', 'test-run.py', 'output.txt', 'stamp-676.fits']


Restricted:

In [10]:
print(runFilterRestricted(test_glob))

ImportError: __import__ not found

#### pathlib

In [11]:
test_pathlib = """
def run_filter():
    from pathlib import Path

    my_file = Path("/home/respy/allowed.py")
    if my_file.is_file():
        return True
    else:
        return False
"""

Unrestricted:

In [12]:
print(runFilterUnrestricted(test_pathlib))

True


Restricted:

In [13]:
print(runFilterRestricted(test_pathlib))

ImportError: __import__ not found

#### subprocess

In [14]:
test_subprocess = """
def run_filter():
    bashCommand = "ls /"
    import subprocess
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    return output
"""

Unrestricted:

In [15]:
print(runFilterUnrestricted(test_subprocess))

b'anaconda-post.log\nbin\nboot\ndev\netc\nhome\nlib\nlib64\nmedia\nmnt\nopt\npacker-files\nproc\nroot\nrun\nsbin\nsrv\nsys\ntmp\nusr\nvar\n'


Restricted:

In [16]:
print(runFilterRestricted(test_subprocess))

ImportError: __import__ not found

### Generally, importing modules does not work, which takes care of packages such as sys, os, subprocess, glob, multiprocessing, threading, psutil (killing processes, etc.), and others that may be problematic.

#### exec

In [17]:
test_execfile = """
def run_filter():
    return exec(open('hello-world.py').read())
"""

Unrestricted:

In [18]:
runFilterUnrestricted(test_execfile)

hello world


Restricted:

In [19]:
print(runFilterRestricted(test_execfile))

SyntaxError: ('Line 3: Exec calls are not allowed.',) (<string>)

#### reading a file

In [20]:
test_openfile = """
def run_filter():
    with open('hello-world.py') as f:
        stuff = f.read()
    return stuff
"""

Unrestricted:

In [21]:
print(runFilterUnrestricted(test_openfile))

def run_filter():
    return 'hello world'

print(run_filter())



Restricted:

In [22]:
print(runFilterRestricted(test_openfile))

NameError: name 'open' is not defined

#### writing a file

In [23]:
test_writedisk = """
def run_filter():
    with open('output.txt', 'a') as f:
        f.write('Hello')
"""

Unrestricted (output file written):

In [24]:
print(runFilterUnrestricted(test_writedisk))

None


Restricted:

In [25]:
print(runFilterRestricted(test_writedisk))

NameError: name 'open' is not defined

#### write to screen

In [26]:
test_writescreen = """
def run_filter():
    print('hello world')
"""

Unrestricted:

In [27]:
runFilterUnrestricted(test_writescreen)

hello world


Restricted:

In [28]:
print(runFilterRestricted(test_writescreen))

/opt/lsst/software/stack/python/miniconda3-4.3.21/lib/python3.6/site-packages/RestrictedPython/compile.py:202: SyntaxWarning: Line 2: Prints, but never reads 'printed' variable.
  SyntaxWarning


NameError: name '_print_' is not defined

## Capabilities we may want to allow even when running in restricted mode

* math
* random
* numpy
* scipy
* pandas
* sklearn
* astropy packages (for stamps)
* lsst packages (for stamps)

#### math

In [29]:
test_math= """
def run_filter():
    return(math.floor(4.8))
"""

In [30]:
print(runFilterRestricted(test_math))

4


#### random

In [31]:
test_random = """
def run_filter():
    return(random.random())
"""

In [32]:
print(runFilterRestricted(test_random))

0.6396843271259767


#### numpy

In [33]:
test_numpy = """
def run_filter():
    return(numpy.mean([1,2,3,4,5]))
"""

In [34]:
print(runFilterRestricted(test_numpy))

3.0


#### scipy

In [35]:
test_scipy = """
def run_filter():
    f = lambda x: -scipy.special.jv(3, x)
    sol = scipy.optimize.minimize(f, 1.0)
    return sol
"""

In [36]:
print(runFilterRestricted(test_scipy))

      fun: -0.43439442684052315
 hess_inv: array([[ 56.41708509]])
      jac: array([  3.35276127e-08])
  message: 'Optimization terminated successfully.'
     nfev: 27
      nit: 2
     njev: 9
   status: 0
  success: True
        x: array([ 4.20118907])


#### pandas

In [37]:
test_pandas = """
def run_filter():
    d = {'col1': [1, 2], 'col2': [3, 4]}
    df = pandas.DataFrame(data=d)
    return df
"""

In [38]:
print(runFilterRestricted(test_pandas))

   col1  col2
0     1     3
1     2     4


### Running functions within pandas let's you circumvent restricted operations like reading and writing to disk. We may or may not want to restrict this.  

In [39]:
test_pandasread = """
def run_filter():
    df = pandas.read_csv('allowed.py')
    return df
"""

In [40]:
print(runFilterRestricted(test_pandasread))

                                        import numpy
0                                       import scipy
1                                      import pandas
2                                      import random
3                                     import sklearn
4                                 import sklearn.svm
5                                     import astropy
6                                  import astropy.io
7                                        import lsst
8                                    import lsst.afw
9                               import lsst.afw.math
10                             import lsst.afw.image
11                                lsst_builtins = {}
12                    lsst_builtins['numpy'] = numpy
13                    lsst_builtins['scipy'] = scipy
14                  lsst_builtins['pandas'] = pandas
15                  lsst_builtins['random'] = random
16                lsst_builtins['sklearn'] = sklearn
17        lsst_builtins['sklearn.svm'] = sklea

In [41]:
test_pandaswrite = """
def run_filter():
    d = {'col1': [1, 2], 'col2': [3, 4]}
    df = pandas.DataFrame(data=d)
    df.to_csv('pandas-test.csv')
"""

In [42]:
print(runFilterRestricted(test_pandaswrite))

None


#### sklearn

Show the following function is valid unrestricted:

In [43]:
test_sklearn_unrestricted = """
def run_filter():
    import sklearn.svm
    clf = sklearn.svm.SVC(gamma=0.001, C=100.)
    return clf
"""

In [44]:
print(runFilterUnrestricted(test_sklearn_unrestricted))

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [45]:
test_sklearn = """
def run_filter():
    clf = sklearn.svm.SVC(gamma=0.001, C=100.)
    return clf
"""

In [46]:
print(runFilterRestricted(test_sklearn))

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


#### astropy

Running unrestricted (allowing imports) shows the following function is valid:

In [47]:
test_astropy_unrestricted = """
def run_filter():
    from astropy.io import fits
    stamp = 'stamp-676.fits'
    hdul = fits.open(stamp)
    return hdul.info()
"""

In [48]:
print(runFilterUnrestricted(test_astropy_unrestricted))

Filename: stamp-676.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  IMAGE         1 PrimaryHDU      35   (31, 31)   float32   
  1  MASK          1 ImageHDU        38   (31, 31)   int16 (rescales to uint16)   
  2  VARIANCE      1 ImageHDU        36   (31, 31)   float32   
  3  PSF           1 ImageHDU         9   (31, 31)   float64   
None


Running restricted with astropy.io imported in lsst_builtins:

In [49]:
test_astropy = """
def run_filter():
    stamp = 'stamp-676.fits'
    hdul = astropy.io.fits.open(stamp)
    return hdul.info()
"""

In [50]:
print(runFilterRestricted(test_astropy))

Filename: stamp-676.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  IMAGE         1 PrimaryHDU      35   (31, 31)   float32   
  1  MASK          1 ImageHDU        38   (31, 31)   int16 (rescales to uint16)   
  2  VARIANCE      1 ImageHDU        36   (31, 31)   float32   
  3  PSF           1 ImageHDU         9   (31, 31)   float64   
None


#### lsst 

In [51]:
test_lsst = """
def run_filter():
    exposure = lsst.afw.image.ExposureF('stamp-676.fits')
    maskedImage = exposure.getMaskedImage()
    statFlags = lsst.afw.math.NPOINT | lsst.afw.math.MEAN | lsst.afw.math.STDEV | lsst.afw.math.MAX | lsst.afw.math.MIN | lsst.afw.math.ERRORS
    imageStatistics = lsst.afw.math.makeStatistics(maskedImage, statFlags)
    mean = imageStatistics.getResult(lsst.afw.math.MEAN)
    return mean
"""

In [52]:
print((runFilterRestricted(test_lsst)))


(31.265613677567288, 3.883924127850158)


## Summary

RestrictedPython can restrict the main basic modules used for viewing and manipulating files on disk and killing running processes.  Using higher level packages such as pandas can circumvent writes and reads by using included package builtins (such as pandas.to_csv, astropy, lsst packages, e.g.), so whitelisted modules should be carefully considered depending on what we deem safe and allowable for filters.